<a href="https://colab.research.google.com/github/Dvvlol/Alura_Desafio_Api_Davi/blob/main/Desafio_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando
import gtts as GT
from playsound import playsound as PS
import googlemaps as gmaps
import google.generativeai as gnai

# Chave API do Google Maps e google.generativeai

GOOGLE_API_KEY="INSIRA_SUA_API_KEY"

# Números de Emergência
emergencias = {
    "Corpo de Bombeiros": "193",
    "Defesa Civil": "199",
}

# classe AlertaRS
class AlertaRS:
    """
    Attributes:
        nome: Person's name
        localizacao: Person's location
        altura_inundacao: Flood height
        animais: Whether the person has animals
        animais_quantidade: Number of animals
        outras_pessoas: Whether there are other people
        outras_pessoas_quantidade: Number of other people
        gravidade: Flood severity
    """

    def __init__(self):
        print("Olá eu sou o AlertaRS, estou aqui para ajudá-lo")
        self.nome = input("Qual o seu nome? ")
        self.localizacao = input("Qual o seu endereço? ")
        self.altura_inundacao = self.obter_numero(
            "Qual é a altura da inundação em metros (ex.: 1.0) ou centímetros (ex.: 0.3) onde você está? "
        )
        self.desabamento_inundacao = input("Já existe algum desabamento em volta de sua casa? ")
        self.animais = self.obter_resposta_sim_nao("Você tem animais domésticos? (s/n) ")
        if self.animais == "s":
            self.animais_quantidade = self.obter_numero("Quantos? ")
        else:
            self.animais_quantidade = 0
        self.outras_pessoas = self.obter_resposta_sim_nao("Há outras pessoas com você? (s/n) ")
        if self.outras_pessoas == "s":
            self.outras_pessoas_quantidade = self.obter_numero("Quantas? ")
        else:
            self.outras_pessoas_quantidade = 0
        self.desabrigado_inundacao = input("Pela gravidade da situação, você considera-se já uma pessoa desabrigada? ")

    def avaliar_gravidade(self):
        # Dicionário de gravidade de inundação
        gravidade_dict = {
            (0.10, 0.49): "Pouca gravidade (água na canela)",
            (0.50, 0.79): "Atenção (água no joelho)",
            (0.80, 1.59): "Procure abrigo imediatamente (água na cintura)",
            (1.60, 1.79): "Estado de emergência (água na altura do peito)",
            (1.80, 40.0): "Extremamente grave (risco de afogamento)",
        }
        for limites, gravidade in gravidade_dict.items():
            if limites[0] <= self.altura_inundacao <= limites[1]:
                self.gravidade = gravidade
                break

    def emitir_instrucoes(self):
        print(f"Olá {self.nome}, a gravidade da inundação em {self.localizacao} é {self.gravidade}.")
        if self.gravidade == "Pouca gravidade (água na canela)":
            print("Você pode ficar em sua casa, mas monitore a situação.")
        elif self.gravidade == "Atenção (água no joelho)":
            print("Procure um lugar mais alto para se abrigar imediatamente.")
        else:
            print(
                f"Conectando aos serviçoes de emergência! ({', '.join(emergencias.values())}) Atenção! Procure um lugar seguro imediatamente!"
            )

    def checar_desabrigo(self):
        desabrigado = input("Você consegue se locomover? (s/n) ")
        if desabrigado == "s":
            gmaps_client = gmaps.Client(key=SECRET_KEY_MAPS)
            servicos = gmaps_client.places_nearby(
                query="Ajuda - Serviços sociais", location=self.localizacao
            )
            for servico in servicos["results"]:
                print(servico["name"] + ": " + servico["vicinity"])

    def mostrar_rotas_fuga(self):
        gmaps_client = gmaps.Client(key=SECRET_KEY_MAPS)
        rotas = gmaps_client.directions(self.localizacao, "Local seguro", mode="driving")
        for rota in rotas:
            print(rota["legs"][0]["steps"][0]["html_instructions"])

    def verificar_previsao(self):
        gmaps_client = gmaps.Client(key=SECRET_KEY_MAPS)
        previsao = gmaps_client.weather(self.localizacao)
        for tempo in previsao[0]["weather"]:
            if tempo["main"] in ["Rain", "Thunderstorm"]:
                # Usando a API Generative AI para gerar um alerta de áudio personalizado
                 # (Adaptando o prompt de acordo a necessidade)
                prompt = f"Crie um alerta sonoro de inundação para {self.nome} em {self.localizacao} com a gravidade {self.gravidade}."
                audio_content = gnai.generate_audio(prompt)
               # Reproduzindo o áudio gerado
                 # (Implementando a reprodução de acordo com seu ambiente)
                # ...

    def conversar(self):
        print("As equipes de Emergência estão a caminho.")
        print("Mantenha-se calmo, procure a parte mais alta do local onde você está.")
        print("Faça uma oração, Acalme as pessoas e os animais que estão com você, que tudo ficará bem. ")
        self.avaliar_gravidade()
        self.emitir_instrucoes()
        self.checar_desabrigo()
        self.mostrar_rotas_fuga()
        self.verificar_previsao()

    def obter_numero(self, prompt):
        while True:
            try:
                valor = float(input(prompt))
                return valor
            except ValueError:
                print("Por favor, digite um número válido.")

    def obter_resposta_sim_nao(self, prompt):
        while True:
            resposta = input(prompt).lower()
            if resposta in ["s", "n"]:
                return resposta
            else:
                print("Por favor, responda com 's' ou 'n'.")

# Run the chatbot
alerta = AlertaRS()
alerta.conversar()